In [1]:
# 라이브러리 임포트
import os
import pandas as pd
import chardet

# 사용자 정의함수 경로 설정
import sys
sys.path.append("G:/내 드라이브/Source/_Custom_Function")
# 사용자 정의함수 불러오기

import time

import ExcelFile_Merge as em
import ExcelFile_Read as er
import ExcelFile_EDA as ee

## 기본 데이터 불러오기

In [4]:
start_time = time.time()
df_BikeRent = pd.read_parquet('G:\\내 드라이브\\DataSet\\_파킷 파일\\서울특별시 공공자전거 대여이력 정보\\서울특별시 공공자전거 대여이력 정보_2021_preprocessed.parquet')
print("read_csv time :", time.time() - start_time)

read_csv time : 15.216501712799072


In [8]:
df_BikeRent.head(3)

,자전거번호,대여대여소번호,대여대여소명,대여거치대,반납대여소번호,반납대여소명,반납거치대,이용시간(분),이용거리(M),대여일자,대여시간,반납일자,반납시간,대여시간차,이용시간-대여시간차,이용속도(km/h)
2,SPB-50952,00003,중랑센터,0,00668,서울축산농협(장안지점),0,11.0,1853.99,2021-01-13,21:02:05,2021-01-13,21:14:04,0 days 00:11:59,-0.983333,10.112673
4,SPB-52380,00003,중랑센터,0,00540,군자역 7번출구 베스트샵 앞,0,13.0,2291.05,2021-01-18,18:02:08,2021-01-18,18:15:22,0 days 00:13:14,-0.233333,10.574077
5,SPB-30541,00003,중랑센터,0,00594,중랑물재생센터(서울새활용플라자),0,2.0,0.00,2021-01-19,18:03:00,2021-01-19,18:05:28,0 days 00:02:28,-0.466667,0.000000


In [16]:
df_BikeRent.shape[0]

27493559

In [18]:
start_time = time.time()
df_BikeBroken = pd.read_parquet('G:\\내 드라이브\\DataSet\\_최종 병합 파일\\서울시 공공자전거 고장신고 내역_15-24.06.parquet')
print("read_csv time :", time.time() - start_time)

read_csv time : 0.3795597553253174


In [20]:
df_BikeBroken.head(3)

,자전거번호,고장구분,등록일자,등록시간
0,SPB-00108,체인,2015-12-04,07:41:01
1,SPB-00210,체인,2015-12-07,12:23:33
2,SPB-00035,기타,2015-12-09,17:07:00


In [22]:
df_BikeBroken.shape[0]

767715

---
### df_BikeRent 의 데이터를 조건에 재조회

In [27]:
# 1. 조건에 따라 필터링: 대여시간차가 5분 미만, 대여 대여소 번호와 반납 대여소 번호가 동일한 데이터만 선택
df_BikeRent_2 = df_BikeRent[(df_BikeRent['대여시간차'] < pd.Timedelta(minutes=5)) & (df_BikeRent['대여대여소번호'] == df_BikeRent['반납대여소번호'])].copy()

# 대여일자를 날짜 형식으로 변환 (필요한 경우)
df_BikeRent_2['대여일자'] = pd.to_datetime(df_BikeRent_2['대여일자'])

In [34]:
df_BikeRent_2.shape[0]

164740

In [36]:
df_BikeRent_2.head(3)

,자전거번호,대여대여소번호,대여대여소명,대여거치대,반납대여소번호,반납대여소명,반납거치대,이용시간(분),이용거리(M),대여일자,대여시간,반납일자,반납시간,대여시간차,이용시간-대여시간차,이용속도(km/h)
22,SPB-53909,00010,위트콤,0,00010,위트콤,0,2.0,0.0,2021-01-05,10:58:33,2021-01-05,11:01:13,0 days 00:02:40,-0.666667,0.0
23,SPB-36252,00010,위트콤,0,00010,위트콤,0,2.0,0.0,2021-01-05,11:10:36,2021-01-05,11:13:16,0 days 00:02:40,-0.666667,0.0
24,SPB-47049,00010,위트콤,0,00010,위트콤,0,3.0,0.0,2021-01-05,13:46:33,2021-01-05,13:50:10,0 days 00:03:37,-0.616667,0.0


---
### '자전거번호' 기준으로 각 데이터 사이별로 '대여일자'간의 차이를 계산하여 '대여사이기간' 컬럼 생성

In [40]:
# 2. 대여일자 간의 차이를 계산하여 대여사이기간 구하기
all_intervals = []

for bike_id, group in df_BikeRent_2.groupby('자전거번호'):
    group = group.sort_values('대여일자').reset_index(drop=True) # 대여일자 기준으로 재정렬, 인덱스는 처음부터 재 설정
    # 현재 행과 이전 행 간의 차이를 계산합니다. 이 경우, '대여일자' 열에 대해 연속된 날짜 간의 차이를 계산합니다.
    group['대여사이기간'] = group['대여일자'].diff()
    
    # 대여사이기간 데이터만 저장
    for i in range(1, len(group)):
        start_date = group['대여일자'].iloc[i - 1] 
        end_date = group['대여일자'].iloc[i]
        interval = group['대여사이기간'].iloc[i]
        
        all_intervals.append({
            '자전거번호': bike_id,
            '기간 시작일': start_date,
            '기간 종료일': end_date,
            '대여사이기간': interval
        })

# DataFrame으로 변환
df_BikeRent_3 = pd.DataFrame(all_intervals)

In [42]:
df_BikeRent_3.shape[0]

137234

In [46]:
df_BikeRent_3.head(3)

,자전거번호,기간 시작일,기간 종료일,대여사이기간
0,SPB-01957,2021-02-19,2021-02-19,0 days
1,SPB-01957,2021-02-19,2021-02-19,0 days
2,SPB-01957,2021-02-19,2021-02-19,0 days


### 중복제거

In [48]:
df_BikeRent_3 = df_BikeRent_3.drop_duplicates()

In [50]:
df_BikeRent_3.shape[0]

135973

In [52]:
df_BikeRent_3.head()

,자전거번호,기간 시작일,기간 종료일,대여사이기간
0,SPB-01957,2021-02-19,2021-02-19,0 days
3,SPB-01957,2021-02-19,2021-02-26,7 days
4,SPB-01957,2021-02-26,2021-03-04,6 days
5,SPB-01957,2021-03-04,2021-03-15,11 days
6,SPB-01957,2021-03-15,2021-05-26,72 days


---
### 대여사이기간 - 이상치 제거

In [54]:
df_BikeRent_4 = df_BikeRent_3

In [44]:
# 3. 대여사이기간 이상치 설정 및 필터링
Q1 = intervals_df['대여사이기간'].quantile(0.25)
Q3 = intervals_df['대여사이기간'].quantile(0.75)
IQR = Q3 - Q1
outlier_threshold = Q3 + 1.5 * IQR
outlier_intervals_df = intervals_df[intervals_df['대여사이기간'] > outlier_threshold]

# 이상치로 긴 대여 사이 기간을 가진 자전거 번호
outlier_bike_ids = outlier_intervals_df['자전거번호'].unique()

In [56]:
outlier_threshold

Timedelta('109 days 12:00:00')

In [42]:
outlier_intervals_df.shape[0]

135973

In [44]:
outlier_intervals_df.head()

,자전거번호,기간 시작일,기간 종료일,대여사이기간
0,SPB-01957,2021-02-19,2021-02-19,0 days
3,SPB-01957,2021-02-19,2021-02-26,7 days
4,SPB-01957,2021-02-26,2021-03-04,6 days
5,SPB-01957,2021-03-04,2021-03-15,11 days
6,SPB-01957,2021-03-15,2021-05-26,72 days


In [54]:
len(outlier_bike_ids)

6706

---
### 자전거 대역이력과 고장장신고 데이터를 비교

In [56]:
df_BikeRent_4.head(2)

,자전거번호,기간 시작일,기간 종료일,대여사이기간
0,SPB-01957,2021-02-19,2021-02-19,0 days
3,SPB-01957,2021-02-19,2021-02-26,7 days


In [58]:
df_BikeBroken.head(2)

,자전거번호,고장구분,등록일자,등록시간
0,SPB-00108,체인,2015-12-04,07:41:01
1,SPB-00210,체인,2015-12-07,12:23:33


In [120]:
# 4. 상관성 확인: df1에서 해당 자전거 번호의 등록일자가 대여사이기간 내에 있는지 확인
df_BikeBroken_2 = df_BikeBroken[df_BikeBroken['자전거번호'].isin(outlier_bike_ids)].copy()
df_BikeBroken_2['등록일자'] = pd.to_datetime(df_BikeBroken_2['등록일자'])

In [122]:
df_BikeBroken_2.head()

,자전거번호,고장구분,등록일자,등록시간
24873,SPB-17759,타이어,2018-09-17,00:23:30
39449,SPB-17759,체인,2019-03-04,00:19:56
39450,SPB-17759,기타,2019-03-04,00:19:56
45474,SPB-17759,체인,2019-04-24,18:26:36
75451,SPB-17759,단말기,2019-07-24,21:13:26


In [62]:
print(f'자전거 대여이력 데이터 : {df_BikeRent_4.shape[0]}')
print(f'자전거 고장신고 데이터 : {df_BikeBroken.shape[0]}')

자전거 대여이력 데이터 : 135973
자전거 고장신고 데이터 : 767715


In [64]:
df_BikeRent_4.head(2)

,자전거번호,기간 시작일,기간 종료일,대여사이기간
0,SPB-01957,2021-02-19,2021-02-19,0 days
3,SPB-01957,2021-02-19,2021-02-26,7 days


In [72]:
print(df_BikeRent_4['기간 시작일'].min())
print(df_BikeRent_4['기간 시작일'].max())

2021-01-01 00:00:00
2021-12-31 00:00:00


In [74]:
print(df_BikeBroken['등록일자'].min())
print(df_BikeBroken['등록일자'].max())

2015-12-04
2024-06-30


---
### 2021년 고장신고 데이터만 조회

In [84]:
df_BikeBroken = df_BikeBroken[df_BikeBroken['등록일자'].between('2021-01-01', '2021-12-31')]

In [86]:
df_BikeBroken.shape[0]

150938

In [88]:
df_BikeBroken.head(2)

,자전거번호,고장구분,등록일자,등록시간
186932,SPB-41936,기타,2021-01-01,0:44
186933,SPB-42181,타이어,2021-01-01,2:42


In [90]:
correlated_bikes = []

# "자전거 대여이력" 데이터만큼 반복
for _, row in df_BikeRent_4.iterrows():
    bike_id = row['자전거번호']
    start_date = row['기간 시작일']
    end_date = row['기간 종료일']

    # "자전거 대여이력"의 등록일자 사이에, "고장신고 내역"이 있는 경우 체크
    date_in_range = df_BikeBroken[
        (df_BikeBroken['자전거번호'] == bike_id) &
        (df_BikeBroken['등록일자'] >= start_date) &
        (df_BikeBroken['등록일자'] <= end_date)
    ]
    
    if not date_in_range.empty:
        correlated_bikes.append({
            '자전거번호': bike_id,
            '최초_대여시작': start_date,
            '마지막_대여시작': end_date,
            '대여사이기간': row['대여사이기간'],
            '최초_고장신고': date_in_range.iloc[0]['등록일자'],
            '마지막_고장신고': date_in_range.iloc[-1]['등록일자'],
            '전체_고장신고': date_in_range['등록일자'].tolist(),
            '횟수_고장신고' : len(date_in_range)
        })

# 결과를 데이터프레임으로 변환
final_result_df = pd.DataFrame(correlated_bikes)

TypeError: '<=' not supported between instances of 'Timestamp' and 'str'

In [126]:
final_result_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13942 entries, 0 to 13941
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype          
---  ------    --------------  -----          
 0   자전거번호     13942 non-null  object         
 1   기간 시작일    13942 non-null  datetime64[ns] 
 2   기간 종료일    13942 non-null  datetime64[ns] 
 3   대여사이기간    13942 non-null  timedelta64[ns]
 4   최조_등록일자   13942 non-null  datetime64[ns] 
 5   마지막_등록일자  13942 non-null  datetime64[ns] 
 6   등록일자      13942 non-null  object         
dtypes: datetime64[ns](4), object(2), timedelta64[ns](1)
memory usage: 762.6+ KB


In [128]:
final_result_df

,자전거번호,기간 시작일,기간 종료일,대여사이기간,최조_등록일자,마지막_등록일자,등록일자
0,SPB-30036,2021-01-13,2021-04-28,105 days,2021-04-04,2021-04-04,[2021-04-04 00:00:00]
1,SPB-30036,2021-05-22,2021-07-04,43 days,2021-05-24,2021-05-24,[2021-05-24 00:00:00]
2,SPB-30036,2021-07-04,2021-12-16,165 days,2021-09-17,2021-12-02,"[2021-09-17 00:00:00, 2021-12-02 00:00:00, 202..."
3,SPB-30044,2021-02-27,2021-03-07,8 days,2021-02-28,2021-03-07,"[2021-02-28 00:00:00, 2021-03-07 00:00:00]"
4,SPB-30044,2021-03-07,2021-03-07,0 days,2021-03-07,2021-03-07,[2021-03-07 00:00:00]


In [130]:
final_result_df['고장사이기간'] = final_result_df['마지막_등록일자'] - final_result_df['최조_등록일자']

In [152]:
final_result_df['고장신고_대기'] = final_result_df['최조_등록일자'] - final_result_df['기간 시작일']

In [160]:
final_result_df.columns = ['자전거번호', '최초_대여시작', '마지막_대여시작', '대여사이기간', '최초_고장신고', '마지막_고장신고', '비교_고장신고', '전체_고장신고', '등록일자']

In [168]:
# 결과 출력
_columns = ['자전거번호', '최초_대여시작', '마지막_대여시작', '대여사이기간', '최초_고장신고', '마지막_고장신고', '비교_고장신고', '전체_고장신고', '등록일자']
display(final_result_df[_columns].head(5))

,자전거번호,최초_대여시작,마지막_대여시작,대여사이기간,최초_고장신고,마지막_고장신고,비교_고장신고,전체_고장신고,등록일자
0,SPB-30036,2021-01-13,2021-04-28,105 days,2021-04-04,2021-04-04,[2021-04-04 00:00:00],0 days,81 days
1,SPB-30036,2021-05-22,2021-07-04,43 days,2021-05-24,2021-05-24,[2021-05-24 00:00:00],0 days,2 days
2,SPB-30036,2021-07-04,2021-12-16,165 days,2021-09-17,2021-12-02,"[2021-09-17 00:00:00, 2021-12-02 00:00:00, 202...",76 days,75 days
3,SPB-30044,2021-02-27,2021-03-07,8 days,2021-02-28,2021-03-07,"[2021-02-28 00:00:00, 2021-03-07 00:00:00]",7 days,1 days
4,SPB-30044,2021-03-07,2021-03-07,0 days,2021-03-07,2021-03-07,[2021-03-07 00:00:00],0 days,0 days


## 대여일자 와 고장신고일자 와의 관계

### 대여사이기간

### 대여사이기간

In [158]:
print(final_result_df['고장신고_대기'].min())
print(final_result_df['고장신고_대기'].mean())
print(final_result_df['고장신고_대기'].max())

0 days 00:00:00
27 days 13:50:49.490747382
269 days 00:00:00


In [140]:
final_result_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13942 entries, 0 to 13941
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype          
---  ------    --------------  -----          
 0   자전거번호     13942 non-null  object         
 1   기간 시작일    13942 non-null  datetime64[ns] 
 2   기간 종료일    13942 non-null  datetime64[ns] 
 3   대여사이기간    13942 non-null  timedelta64[ns]
 4   최조_등록일자   13942 non-null  datetime64[ns] 
 5   마지막_등록일자  13942 non-null  datetime64[ns] 
 6   등록일자      13942 non-null  object         
 7   비교_등록일자   13942 non-null  timedelta64[ns]
dtypes: datetime64[ns](4), object(2), timedelta64[ns](2)
memory usage: 871.5+ KB


In [134]:
final_result_df.shape[0]

13942